# Tutoriel de prise en main de PythonMIP (partie 2)

Dans ce notebook, on vous explique au travers d'un exemple notamment comment utiliser le paquet _PythonMIP_ pour résoudre des programmes linéaires (en nombres entiers) **plus complexes** avec lecture des données dans un fichier.

Nous allons écrire un code permettant de résoudre le problème de localisation d'entrepôts sous contraintes de capacité. 

Dans ce problème, on dispose d'un ensemble $\mathcal{W}$ d'entrepôts potentiels et d'un ensemble $\mathcal{J}$ de clients. Un coût $f_i$ est associé à l'ouverture d'un entrepôt $i\in \mathcal{W}$. Chaque entrepôt $i$ possède une capacité $Q_i$ (stockage, fabrication,...) et chaque client $j\in \mathcal{J}$ a une demande $d_j>0$. Un coût $c_{ji}$ est associé à chaque unité de la demande d'un client $j$ satisfaite par l'entrepôt $i$. Il s'agit de décider de l'ouverture d'entrepôts parmi ceux de $\mathcal{W}$ et de l'affectation des clients aux entrepôts qui auront été ouverts. L'objectif est de minimiser la somme des coûts d'ouverture et d'affectation. Une solution réalisable du problème doit satisfaire les contraintes suivantes :
* La totalité de la demande de chaque client doit être affectée aux entrepôts ouverts
* La somme des demandes des clients affectés à un entrepôt ouivert $i\in \mathcal{W}$ doit être plus petite ou égale que sa capacité $Q_i$

Ce problème peut se modéliser en utilisant la programmation linéaire en nombres entiers. Soit $z_i$ la variable binaire égale à 1 si et seulement si l'entrepôt $i\in \mathcal{W}$ est ouvert. Soit $y_{ji}$ la variable continue indiquant la fraction de la demande du client $j\in \mathcal{J}$ servie par l'entrepôt $i$. On peut écrire le programme linéaire en nombres entiers suivant :

$$\begin{array}{rlll}
\min & \sum\limits_{i\in \mathcal{W}}f_iz_i + \sum\limits_{j\in \mathcal{J}}\sum\limits_{i\in \mathcal{W}}c_{ji}y_{ji} & & (1) \\
\text{s.c.} & \sum\limits_{i\in \mathcal{W}}y_{ji} = 1 & \forall j\in \mathcal{J} & (2)\\
 & \sum_\limits{j\in \mathcal{J}}d_jy_{ji} \leq Q_iz_i & \forall i\in \mathcal{W} & (3) \\
 & z_i \in \{0,1\} & \forall i\in \mathcal{W} & (4) \\
 & y_{ji} \leq 1 & \forall j\in \mathcal{J}, \forall i\in \mathcal{W} & (5) \\
 & y_{ji} \geq  0 & \forall j\in \mathcal{J}, \forall i\in \mathcal{W} & (6)
\end{array}$$

La fonction objectif (1) consiste à minimiser la somme des coûts d'ouverture des entrepôts et des coûts d'affectations des clients. Les contraintes (2) imposent la satisfaction totale de la demande de chacun des clients. Les contraintes (3) imposent le respect des capacités de chacun des entrepôts. Enfin, les contraintes (4) et (5) définissent le domaine des variables.




## Lire les données dans un fichier

Voyons comme lire les données d'un fichier. Ces données sont organisés dans le fichier _cap41.txt_ situé dans le dossier _data\_CWFL_ de la manière suivante :  
_#Taille des données (ligne 1)_  
$\vert \mathcal{W}\vert$ $\vert \mathcal{J}\vert$  
_#Pour chaque entrepôt $i\in \mathcal{W}$ (ligne 2 à $1+\vert \mathcal{W}\vert$)_  
$Q_i$ $f_i$  
_#Pour chaque client $j\in \mathcal{J}$(ligne $2+\vert \mathcal{W}\vert$ à $2+\vert \mathcal{W}\vert+2\vert \mathcal{J}\vert$)_  
$d_j$  
$c_{j1}$ $c_{j2}$ ... $c_{j\vert \mathcal{W}\vert}$

Nous considérons que les données seront toujours données sous ce format. Le code suivant permettra alors de résoudre plusieurs jeux de données en n'ayant qu'à modifier le nom du fichier à lire.

In [1]:
import re
datafileName = '../data_CWFL/cap41.txt' #chemin relatif vers le fichier (l'utilisation .. permet de revenir au dossier parent)
with open(datafileName, "r") as file:  #ouvre le fichier, le ferme automatiquement à la fin et gère les exceptions
    line = file.readline()  # lit la 1ère ligne
    lineTab = line.split()  # sépare les éléments de la ligne dans un tableau en utilisant l'espace comme séparateur
    nb_warehouses = int(lineTab[0]) # la valeur de la 1ère case correspond au nombre d'entrepôts (attention de penser à convertir la chaîne de caractère en un entier)
    nb_customers = int(lineTab[1]) # la valeur de la 2ème case correspond au nombre de clients
    capacity = [] # création d'un tableau qui stockera les capacités des entrepôts
    opening_cost = [] # création d'un tableau qui stockera les coûts d'ouverture des entrepôts
    for i in range(nb_warehouses): # pour chaque ligne contenant les informations sur les entrepôts
        line = file.readline()  # lit la ligne suivante
        lineTab = line.split()  # sépare les éléments de la ligne dans un tableau en utilisant l'espace comme séparateur
        capacity.append(int(lineTab[0])) # ajoute l'élément de la 1ère case au tableau qui contient les capacités 
        opening_cost.append(float(lineTab[1])) # ajoute l'élément de la 2ème case au tableau qui contient les coûts d'ouverture 
    demand = [] # création d'un tableau qui stockera les demandes des clients
    assignment_cost = [] # création d'un tableau qui stockera les tableaux de coûts d'affectation aux entrepôts de chaque client
    for j in range(nb_customers): # pour chaque ligne contenant les informations sur les clients
        line = file.readline()  # lit la ligne suivante
        demand.append(int(line.split()[0])) # ajoute l'élément de la 1ère case au tableau qui contient la demande des clients
        cost =[] # création du tableau des coûts d'affectation du client j aux entrepôts
        line = file.readline()  # lit la ligne suivante
        lineTab = line.split()  # sépare les éléments de la ligne dans un tableau en utilisant l'espace comme séparateur
        for i in range(nb_warehouses):
            cost.append(float(lineTab[i])) # ajoute l'élément de la case i au tableau contenant les coûts d'affectation du client j aux entrepôts
        assignment_cost.append(cost) # ajoute le tableau cost au tableau aux entrepôts au tableau contenant les coûts d'affectations de tous les clients au dépôt
        
# Affichage des informations lues
print("Nombre d'entrepôts = ", nb_warehouses)
print("Capacité des entrepôts = ", capacity)
print("Coût d'ouverture des entrepôts = ", opening_cost)
print("Nombre de clients = ", nb_customers)
print("Demande des clients = ", demand)
#for j in range(nb_customers):
#    print(assignment_cost[j])

Nombre d'entrepôts =  16
Capacité des entrepôts =  [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Coût d'ouverture des entrepôts =  [7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0, 0.0, 7500.0, 7500.0, 7500.0, 7500.0, 7500.0]
Nombre de clients =  50
Demande des clients =  [146, 87, 672, 1337, 31, 559, 2370, 1089, 33, 32, 5495, 904, 1466, 143, 615, 564, 226, 3016, 253, 195, 38, 807, 551, 304, 814, 337, 4368, 577, 482, 495, 231, 322, 685, 12912, 325, 366, 3671, 2213, 705, 328, 1681, 1117, 275, 500, 2241, 733, 222, 49, 1464, 222]


## Création et définition du modèle

On commence par créer le modèle (problème) vide en indiquant le sens de l'optimisation (minimisation ou maximisation).

In [4]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *
# Import du paquet time pour calculer le temps de résolution
import time 

# Création du modèle vide 
model = Model(name = "CWFL", sense = mip.MINIMIZE, solver_name="CBC")  # Utilisation de CBC (remplacer par GUROBI pour utiliser cet autre solveur)

On va maintenant créer les variables.

In [5]:
# Création des variables z et y
z = [model.add_var(name="z(" + str(i) + ")", var_type=BINARY) for i in range(nb_warehouses)]
y = [[model.add_var(name="y(" + str(j) + "," + str(i) + ")", lb=0, ub= 1, var_type=CONTINUOUS) for i in range(nb_warehouses)] for j in range(nb_customers)]

#Il aurait été possible de créer les variables comme ceci (cela est équivalent)
#z = []
#for i in range(nb_warehouses):
#    z.append(model.add_var(name="z(" + str(i) + ")", var_type=BINARY))
#y = []
#for j in range(nb_customers):  
#    for i in range(nb_warehouses):
#        y.append(model.add_var(name="y(" + str(j) + "," + str(i) + ")", lb=0, ub= 1, var_type=CONTINUOUS))

On ajoute ensuite la fonction objectif au modèle à la l'aide de la commande ̀`model+=` et la fonction ̀`xsum`

In [6]:
# Ajout de la fonction objectif au modèle
model += (xsum(opening_cost[i] * z[i] for i in range(nb_warehouses))+xsum(assignment_cost[j][i]*y[j][i] for j in range(nb_customers) for i in range(nb_warehouses)) )

On ajoute ensuite les contraintes au modèle à la l'aide de la commande ̀`model+=`et la fonction ̀`xsum`

In [7]:
# Ajout des contraintes au modèle
for j in range(nb_customers):  
    model += (xsum([y[j][i] for i in range(nb_warehouses)]) == 1)  # Contraintes (2)

for i in range(nb_warehouses):  
    model += (xsum([demand[j]*y[j][i] for j in range(nb_customers)]) <= capacity[i]*z[i])  # Contraintes (3)

On peut écrire le modèle `model` que l'on a généré dans un fichier en utilisant la fonction `write`.

In [ ]:
# Ecrire le modèle (ATTENTION ici le modèle est très grand)
#model.write("cwfl.lp") #à décommenter si vous le souhaitez

Il nous reste plus qu'à lancer la résolution de notre programme linéaire `model` en appelant la fonction `optimize`.

In [8]:
# Limitation du nombre de processeurs
model.threads = 1  

# Lancement du chronomètre
start = time.perf_counter()

# Résolution du modèle
status = model.optimize(max_seconds=120)  # temps limite = 120s

# Arrêt du chronomètre et calcul du temps de résolution
runtime = time.perf_counter() - start


Avant de chercher à récupérer le résultat, on vérifie le status de la solution retournée par le solveur.

Il ne reste plus qu'à récupérer la solution obtenue (valeur des variables et de l'objectif) et à l'afficher.

In [9]:
print("\n----------------------------------")
if status == OptimizationStatus.OPTIMAL:
    print("Status de la résolution: OPTIMAL")
elif status == OptimizationStatus.FEASIBLE:
    print("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
    print("Status de la résolution: IRREALISABLE")
elif status == OptimizationStatus.UNBOUNDED:
    print("Status de la résolution: NON BORNE")
    
print("Temps de résolution (s) : ", runtime)
print("----------------------------------")

# Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
if model.num_solutions>0:
    print("Solution calculée")
    print("-> Valeur de la fonction objectif de la solution calculée : ",  model.objective_value)  # ne pas oublier d'arrondir si le coût doit être entier
    print("-> Meilleure borne inférieure sur la valeur de la fonction objectif = ", model.objective_bound)
    for i in range(nb_warehouses):
        if (z[i].x >= 0.5):
            print("- L'entrepôt ",i , " est ouvert [capacité = ", capacity[i], "] et les clients suivants lui sont affectés")
            for j in range(nb_customers):
                if (y[j][i].x >= 1e-4):
                    print("\t Client ",j, " pour ", round(y[j][i].x * 100,1), " % de sa demande -> ",round(y[j][i].x * demand[j],1))
else:
    print("Pas de solution calculée")
print("----------------------------------\n")




----------------------------------
Status de la résolution: OPTIMAL
Temps de résolution (s) :  0.15489186400009203
----------------------------------
Solution calculée
-> Valeur de la fonction objectif de la solution calculée :  1040444.3750000001
-> Meilleure borne inférieure sur la valeur de la fonction objectif =  1040444.3750000001
- L'entrepôt  0  est ouvert [capacité =  5000 ] et les clients suivants lui sont affectés
	 Client  2  pour  100.0  % de sa demande ->  672.0
	 Client  3  pour  100.0  % de sa demande ->  1337.0
	 Client  5  pour  100.0  % de sa demande ->  559.0
	 Client  13  pour  100.0  % de sa demande ->  143.0
	 Client  23  pour  100.0  % de sa demande ->  304.0
	 Client  25  pour  100.0  % de sa demande ->  337.0
	 Client  29  pour  100.0  % de sa demande ->  495.0
	 Client  39  pour  100.0  % de sa demande ->  328.0
	 Client  48  pour  49.7  % de sa demande ->  728.0
- L'entrepôt  1  est ouvert [capacité =  5000 ] et les clients suivants lui sont affectés
	 Clien

## Ecriture du résultat dans un fichier

Si une solution a été calculée, on l'écrit dans un fichier dans le format suivant (arbitraire) facile à lire.  
_#Ligne 1_  
Valeur de la solution  
_#Pour chaque entrepôt ouvert_  
Numéro de l'entrepôt ouvert  
_#Pour chaque client affecté à cet entrepôt_  
Numéro du client et pourcentage de la demande de ce client satisfait par l'entrepôt  

**NB : Si vous écrivez dans le fichier, alors qu'il est ouvert dans un onglet de Jupyter, alors il faut le fermer et le rouvrir pour voir les changements apparaître.**

In [ ]:
if model.num_solutions>0: # Si une solution a été calculée
    solutionfileName = 'solution_cap41.txt'
    with open(solutionfileName, 'w') as file:  #ouvre le fichier, le ferme automatiquement à la fin et gère les exceptions
        file.write(str(model.objective_value)) #Il faut convertir les valeurs numériques en chaîne de caractères
        file.write("\n") #Je passe à la ligne suivante
        for i in range(nb_warehouses):
            if (z[i].x >= 0.5):
                file.write(str(i)) 
                file.write("\n") #Je passe à la ligne suivante
                for j in range(nb_customers):
                    if (y[j][i].x >= 1e-4):
                        file.write(str(j)+" "+str(round(y[j][i].x * 100,2)))
                        file.write("\n") #Je passe à la ligne suivante